In [1]:
import os
from pymilvus import (connections, MilvusClient, utility)
from dotenv import load_dotenv

In [2]:

#/var/tmp/.env
load_dotenv()  # Loads from .env file by default
api_key = os.getenv("GOOGLE_API_KEY")
debug = os.getenv("DEBUG")
pinecone_api_key=os.getenv("PINECONE_API_KEY")
OPENAI_KEY = os.getenv("OPENAI_API_KEY")
MILVUS_USER=os.getenv("MILVUS_USERNAME")
MILVUS_PASSWORD = os.getenv("MILVUS_PASSWORD")
MILVUS_URI = os.getenv("MILVUS_URI")
MILVUS_API_TOKEN = os.getenv("MILVUS_API_TOKEN")

In [3]:
import time
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Milvus
# Removed direct import: from langchain_community.retrievers import MMRRetriever
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from pymilvus import connections, utility
from docx import Document
import os
import pdfplumber # Import pdfplumber

# --- Step 1: Load & Chunk Text ---
pdf_path = r"C:\Gajanan\data\ast_sci_data_tables_sample.pdf"
full_text = ""
try:
    # Use pdfplumber to extract text from the PDF
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            # Extract text from each page and append to full_text
            page_text = page.extract_text()
            if page_text: # Ensure text was extracted from the page
                full_text += page_text + "\n" # Add a newline between pages
    if not full_text.strip(): # Check if any text was actually extracted
         print("Warning: No text extracted from the PDF. Check if the PDF contains extractable text.")
         full_text = "Placeholder text as no text could be extracted from the PDF." # Use placeholder if extraction fails
    print(full_text)
except FileNotFoundError:
    print(f"Error: PDF file not found at the specified path: {pdf_path}")
    full_text = "Placeholder text because the PDF file was not found. Please replace with actual PDF content extraction."
except Exception as e:
    print(f"Error processing PDF file {pdf_path}: {e}")
    full_text = "Placeholder text due to an error during PDF processing."











NATIONAL PARTNERSHIP FOR QUALITY AFTERSCHOOL LEARNING
www.sedl.org/afterschool/toolkits
����������� �������� �������
Tutoring to Enhance Science Skills
Tutoring Two: Learning to Make Data Tables
..............................................................................................
Sample Data for Data Tables
Use these data to create data tables following the Guidelines for Making a Data Table and
Checklist for a Data Table.
Example 1: Pet Survey (GR 2–3)
Ms. Hubert’s afterschool students took a survey of the 600 students at Morales Elementary
School. Students were asked to select their favorite pet from a list of eight animals. Here
are the results.
Lizard 25, Dog 250, Cat 115, Bird 50, Guinea pig 30, Hamster 45, Fish 75,
Ferret 10
Example 2: Electromagnets—Increasing Coils (GR 3–5)
The following data were collected using an electromagnet with a 1.5 volt battery, a switch,
a piece of #20 insulated wire, and a nail. Three trials were run. Safety precautions in
repeating this exp

In [57]:
print(len(full_text))

591891


In [6]:
import fitz  # PyMuPDF
import pytesseract
from PIL import Image
import io
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [7]:
import pdfplumber
import fitz  # PyMuPDF
import pytesseract
from PIL import Image
import io

def extract_text_and_tables(pdf_path, lang='eng', dpi=300, verbose=True):
    """
    Extracts text and tables from a PDF file.
    Uses pdfplumber for digital PDFs, falls back to OCR if no text on page.
    
    Args:
        pdf_path (str): Path to PDF file.
        lang (str): Language(s) for pytesseract OCR.
        dpi (int): Resolution for OCR rendering.
        verbose (bool): Print progress info.

    Returns:
        dict: {
            page_number (int): {
                'text': extracted text (str),
                'tables': list of tables (list of list of rows)
            }
        }
    """
    results = {}
    doc = fitz.open(pdf_path)

    with pdfplumber.open(pdf_path) as plumber_pdf:
        for i in range(len(doc)):
            if verbose:
                print(f"Processing page {i+1}/{len(doc)}...")

            page_data = {'text': '', 'tables': []}

            # Try extracting text + tables using pdfplumber
            plumber_page = plumber_pdf.pages[i]
            text = plumber_page.extract_text()
            tables = plumber_page.extract_tables()

            if text:
                page_data['text'] = text
                page_data['tables'] = tables or []
            else:
                # Fallback to OCR
                page = doc.load_page(i)
                pix = page.get_pixmap(dpi=dpi)
                img = Image.open(io.BytesIO(pix.tobytes("png")))
                ocr_text = pytesseract.image_to_string(img, lang=lang)
                page_data['text'] = ocr_text
                page_data['tables'] = []  # OCR table extraction not handled here

            results[i+1] = page_data

    doc.close()
    return results


In [ ]:
pdf_file = r"C:\Gajanan\data\ast_sci_data_tables_sample.pdf"
data = extract_text_and_tables(pdf_file, lang='eng', dpi=300)
for page_num, content in data.items():
    print(f"\n--- Page {page_num} ---")
    print("Text:")
    print(content['text'])
    if content['tables']:
        print("Tables:")
        for table in content['tables']:
            for row in table:
                print(row)
            print()
    else:
        print("No tables found on this page.")


Processing page 1/2...
Processing page 2/2...
data dict_keys([1, 2])

--- Page 1 ---
Text:
NATIONAL PARTNERSHIP FOR QUALITY AFTERSCHOOL LEARNING
www.sedl.org/afterschool/toolkits
����������� �������� �������
Tutoring to Enhance Science Skills
Tutoring Two: Learning to Make Data Tables
..............................................................................................
Sample Data for Data Tables
Use these data to create data tables following the Guidelines for Making a Data Table and
Checklist for a Data Table.
Example 1: Pet Survey (GR 2–3)
Ms. Hubert’s afterschool students took a survey of the 600 students at Morales Elementary
School. Students were asked to select their favorite pet from a list of eight animals. Here
are the results.
Lizard 25, Dog 250, Cat 115, Bird 50, Guinea pig 30, Hamster 45, Fish 75,
Ferret 10
Example 2: Electromagnets—Increasing Coils (GR 3–5)
The following data were collected using an electromagnet with a 1.5 volt battery, a switch,
a piece of #20 

In [13]:
data

{1: {'text': 'NATIONAL PARTNERSHIP FOR QUALITY AFTERSCHOOL LEARNING\nwww.sedl.org/afterschool/toolkits\n����������� �������� �������\nTutoring to Enhance Science Skills\nTutoring Two: Learning to Make Data Tables\n..............................................................................................\nSample Data for Data Tables\nUse these data to create data tables following the Guidelines for Making a Data Table and\nChecklist for a Data Table.\nExample 1: Pet Survey (GR 2–3)\nMs. Hubert’s afterschool students took a survey of the 600 students at Morales Elementary\nSchool. Students were asked to select their favorite pet from a list of eight animals. Here\nare the results.\nLizard 25, Dog 250, Cat 115, Bird 50, Guinea pig 30, Hamster 45, Fish 75,\nFerret 10\nExample 2: Electromagnets—Increasing Coils (GR 3–5)\nThe following data were collected using an electromagnet with a 1.5 volt battery, a switch,\na piece of #20 insulated wire, and a nail. Three trials were run. Safety pr

In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

def table_to_text(table):
    rows_text = []
    for row in table:
        clean_cells = [cell if cell is not None else '' for cell in row]
        row_text = "\t".join(clean_cells)
        rows_text.append(row_text)
    return "\n".join(rows_text)

def prepare_and_chunk_with_langchain(data, chunk_size=1000, chunk_overlap=100):
    # Combine text + tables into one big string
    combined_text = data['text']
    for i, table in enumerate(data.get('tables', [])):
        combined_text += f"\n\n--- Table {i+1} ---\n"
        combined_text += table_to_text(table)
    print("combined",combined_text)
    # Create LangChain text splitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    # Split text into chunks (returns List[str])
    chunks = text_splitter.split_text(combined_text)

    # Optional: convert chunks to LangChain Document objects (useful for vectorstore ingestion)
    docs = [Document(page_content=chunk) for chunk in chunks]

    return docs


In [19]:
# Prepare and chunk
final_docs = []
for i, info in data.items():
    documents = prepare_and_chunk_with_langchain(info)
    final_docs.extend(documents)

combined NATIONAL PARTNERSHIP FOR QUALITY AFTERSCHOOL LEARNING
www.sedl.org/afterschool/toolkits
����������� �������� �������
Tutoring to Enhance Science Skills
Tutoring Two: Learning to Make Data Tables
..............................................................................................
Sample Data for Data Tables
Use these data to create data tables following the Guidelines for Making a Data Table and
Checklist for a Data Table.
Example 1: Pet Survey (GR 2–3)
Ms. Hubert’s afterschool students took a survey of the 600 students at Morales Elementary
School. Students were asked to select their favorite pet from a list of eight animals. Here
are the results.
Lizard 25, Dog 250, Cat 115, Bird 50, Guinea pig 30, Hamster 45, Fish 75,
Ferret 10
Example 2: Electromagnets—Increasing Coils (GR 3–5)
The following data were collected using an electromagnet with a 1.5 volt battery, a switch,
a piece of #20 insulated wire, and a nail. Three trials were run. Safety precautions in
repeating

In [16]:
final_docs

[Document(metadata={}, page_content='NATIONAL PARTNERSHIP FOR QUALITY AFTERSCHOOL LEARNING\nwww.sedl.org/afterschool/toolkits\n����������� �������� �������\nTutoring to Enhance Science Skills\nTutoring Two: Learning to Make Data Tables\n..............................................................................................\nSample Data for Data Tables\nUse these data to create data tables following the Guidelines for Making a Data Table and\nChecklist for a Data Table.\nExample 1: Pet Survey (GR 2–3)\nMs. Hubert’s afterschool students took a survey of the 600 students at Morales Elementary\nSchool. Students were asked to select their favorite pet from a list of eight animals. Here\nare the results.\nLizard 25, Dog 250, Cat 115, Bird 50, Guinea pig 30, Hamster 45, Fish 75,\nFerret 10\nExample 2: Electromagnets—Increasing Coils (GR 3–5)\nThe following data were collected using an electromagnet with a 1.5 volt battery, a switch,\na piece of #20 insulated wire, and a nail. Three tri

In [21]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=300)
chunks = text_splitter.split_text(full_text)
sum_chunk_total_chars = 0
for chunk in chunks:
    #print(len(chunk))
    sum_chunk_total_chars +=len(chunk)

print(f"Total chunks: {len(chunks)}")

print("sum_chunk_total_chars",sum_chunk_total_chars)
# --- Step 2: Embedding ---
# Ensure 'YOUR_OPENAI_API_KEY' is replaced with an actual valid key or an environment variable
openai_api_key = OPENAI_KEY# Use environment variable or replace
if openai_api_key == OPENAI_KEY:
    print("Warning: Replace 'YOUR_OPENAI_API_KEY' with your actual key or set the OPENAI_API_KEY environment variable.")

embeddings = OpenAIEmbeddings(model="text-embedding-3-small", api_key=openai_api_key,disallowed_special=()) # Use the variable


Total chunks: 6
sum_chunk_total_chars 5759


In [22]:
# --- Step 3: Connect to Milvus and Store Data ---
# Helper to create Milvus index
def store_milvus(chunks, index_name, metric_type, index_type):
    if utility.has_collection(index_name):
        utility.drop_collection(index_name)
    # Ensure the chunks are not empty before creating the collection
    if not chunks:
        print(f"Warning: No chunks to store for index '{index_name}'. Skipping collection creation.")
        return None

    return Milvus.from_texts(
        chunks,
        embedding=embeddings,
        collection_name=index_name,
        connection_args={"uri": "https://in03-c8ed7ba112ba348.serverless.gcp-us-west1.cloud.zilliz.com",
        "token": MILVUS_API_TOKEN},
        index_params={"metric_type": metric_type, "index_type": index_type})


In [65]:
print(Milvus.__doc__)


.. deprecated:: 0.2.0 Use ``:class:`~langchain_milvus.MilvusVectorStore``` instead. It will not be removed until langchain-community==1.0.

`Milvus` vector store.

You need to install `pymilvus` and run Milvus.

See the following documentation for how to run a Milvus instance:
https://milvus.io/docs/install_standalone-docker.md

If looking for a hosted Milvus, take a look at this documentation:
https://zilliz.com/cloud and make use of the Zilliz vectorstore found in
this project.

IF USING L2/IP metric, IT IS HIGHLY SUGGESTED TO NORMALIZE YOUR DATA.

Args:
    embedding_function (Embeddings): Function used to embed the text.
    collection_name (str): Which Milvus collection to use. Defaults to
        "LangChainCollection".
    collection_description (str): The description of the collection. Defaults to
        "".
    collection_properties (Optional[dict[str, any]]): The collection properties.
        Defaults to None.
        If set, will override collection existing properties.
   

In [23]:
milvus_flat = None # Initialize vectorstore variables
milvus_hnsw = None
milvus_ivf = None
milvus_connected = False # Flag to indicate if Milvus connection was successful

# Ensure Milvus server is running and accessible at localhost:19530
try:
    #connections.connect("default", host="localhost", port="19530")
    # Connect to Zilliz cloud using endpoint URI and API key TOKEN.
    #CLUSTER_ENDPOINT="https://in03-xxxx.api.gcp-us-west1.zillizcloud.com:443"
    #CLUSTER_ENDPOINT="https://in03-8bc9fd463236b1a.api.gcp-us-west1.zillizcloud.com:443"
    CLUSTER_ENDPOINT = "https://in03-c8ed7ba112ba348.serverless.gcp-us-west1.cloud.zilliz.com"

    connections.connect(
    alias='default',
    uri=CLUSTER_ENDPOINT,
    token=MILVUS_API_TOKEN

    )
    print("Connected. Existing collections:", utility.list_collections())

    # Check if the server is ready and get collection name.
    print(f"Type of server: {utility.get_server_version()}")
    print("Connected to Milvus")
    milvus_connected = True # Set flag to True on successful connection

    # Create three indexes ONLY if co
    # Create three indexes ONLY if connection is successful
    #milvus_flat = store_milvus(chunks,"flat_index", "L2", IndexType.FLAT)
    milvus_hnsw = store_milvus(chunks, "hnsw_index", "COSINE", "HNSW")
    milvus_ivf = store_milvus(chunks, "ivf_index", "COSINE", "IVF_FLAT")
except Exception as e:
    print(f"Error connecting to Milvus or storing data: {e}")
    # The vectorstore variables will remain None if connection fails

Connected. Existing collections: ['genai_collection', 'flat_index', 'hnsw_index_image', 'hnsw_index', 'ivf_index']
Type of server: Zilliz Cloud Vector Database(Compatible with Milvus 2.4)
Connected to Milvus


In [24]:
# --- Step 4: Retriever + Timer + Accuracy ---
def test_retriever(name, vectorstore, query):
    if vectorstore is None: # Handle cases where collection wasn't created
        print(f"Skipping {name} retriever test as the collection was not created.")
        return []

    # Use search_type argument for different retriever types
    # Ensure the vectorstore object has the as_retriever method
    if hasattr(vectorstore, 'as_retriever'):
        if name == "MMR":
            retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 5, "lambda_mult": 0.5})
        else:
            retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

        start = time.time()
        # Ensure retriever returns list of Documents; handle potential errors
        try:
            docs = retriever.get_relevant_documents(query)
        except Exception as e:
            print(f"Error during {name} retrieval: {e}")
            docs = []
            return docs

        end = time.time()
        print(f"{name} Retriever Time: {end - start:.4f}s")
        # Calculate accuracy if docs are not empty
        # Note: Simple substring match for accuracy is basic;
        # more sophisticated methods are needed for robust evaluation.
        if docs:
            # Ensure document objects have 'page_content' attribute
            hits = sum(1 for d in docs if hasattr(d, 'page_content') and query.lower() in d.page_content.lower())
            accuracy = hits / len(docs)
            print(f"{name} Accuracy: {accuracy:.2%}")
        else:
             print(f"{name} Accuracy: N/A (no documents retrieved)")

        return docs
    else:
        print(f"Error: Vectorstore object for {name} does not have 'as_retriever' method.")
        return []


In [26]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from docx import Document

def run_prompt_tuning_pipeline(query, milvus_flat, milvus_hnsw, milvus_ivf, milvus_connected, openai_api_key):
    docs_flat, docs_hnsw, docs_mmr = [], [], []
    
    if milvus_connected:
        docs_flat = test_retriever("Flat", milvus_flat, query)
        docs_hnsw = test_retriever("HNSW", milvus_hnsw, query)
        print("hnsw Top Docs:", docs_hnsw)
        print("hnsw context",docs_hnsw[1])
        docs_mmr = test_retriever("MMR", milvus_ivf, query)
        print("MMR Top Docs:", docs_mmr)
        print("mmr ivf context",docs_mmr[1])
        print("Skipping retriever tests as Milvus connection failed.")
    
    prompt = PromptTemplate.from_template("""
You are an AI assistant. Use the following retrieved context to answer the question.

Context:
{context}

Question:
{question}
""")
    
    llm = ChatOpenAI(model="gpt-4o", temperature=0.2, api_key=openai_api_key)
    output = "Could not generate answer due to an error or Milvus connection failure."

    if milvus_ivf is not None and milvus_connected:
        try:
            retriever = milvus_ivf.as_retriever(search_type="mmr", search_kwargs={"k": 5, "lambda_mult": 0.5})
            chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type_kwargs={"prompt": prompt})
            output = chain.run(query)
            print("Answer:\n", output)
        except Exception as e:
            print(f"Error running RetrievalQA chain with MMR: {e}")
            output = f"Could not generate answer due to an error: {e}"
    else:
        print("Milvus IVF collection was not initialized or Milvus connection failed.")
        output = "Could not create RetrievalQA chain because the Milvus IVF collection was not initialized or Milvus connection failed."

    # Export to DOCX if successful
    if not output.startswith("Could not"):
        try:
            doc = Document()
            doc.add_heading("LLM Answer", level=1)
            doc.add_paragraph(output)
            doc.save("LLM_Answer_Output.docx")
            print("Output saved to LLM_Answer_Output.docx")
        except Exception as e:
            print(f"Error saving output to DOCX: {e}")
    else:
        print("DOCX file not saved due to an error in generating the answer.")

    return {
        "flat_docs": docs_flat,
        "hnsw_docs": docs_hnsw,
        "mmr_docs": docs_mmr,
        "answer": output
    }


In [27]:
query = "Give me information about ge j47?"
results = run_prompt_tuning_pipeline(
    query=query,
    milvus_flat=milvus_flat,
    milvus_hnsw=milvus_hnsw,
    milvus_ivf=milvus_ivf,
    milvus_connected=milvus_connected,
    openai_api_key=OPENAI_KEY
)


Skipping Flat retriever test as the collection was not created.


C:\Users\Admin\AppData\Local\Temp\ipykernel_8188\108715618.py:18: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)


HNSW Retriever Time: 0.9961s
HNSW Accuracy: 0.00%
hnsw Top Docs: [Document(metadata={'pk': 458312173147910713}, page_content='L’Automobile (FIA), the world’s governing body for motor sport and land speed records,\nrecorded the following land speed records. (Retrieved on February 5, 2006, from\nhttp://www.landspeed.com/lsrinfo.asp.)\nSpeed (mph) Driver Car Engine Date\n407.447 Craig Breedlove Spirit of America GE J47 8/5/63\n413.199 Tom Green Wingfoot Express WE J46 10/2/64\n434.22 Art Arfons Green Monster GE J79 10/5/64\n468.719 Craig Breedlove Spirit of America GE J79 10/13/64\n526.277 Craig Breedlove Spirit of America GE J79 10/15/65\n536.712 Art Arfons Green Monster GE J79 10/27/65\n555.127 Craig Breedlove Spirit of America, Sonic 1 GE J79 11/2/65\n576.553 Art Arfons Green Monster GE J79 11/7/65\n600.601 Craig Breedlove Spirit of America, Sonic 1 GE J79 11/15/65\n622.407 Gary Gabelich Blue Flame Rocket 10/23/70\n633.468 Richard Noble Thrust 2 RR RG 146 10/4/83\n763.035 Andy Green Th

C:\Users\Admin\AppData\Local\Temp\ipykernel_8188\515078519.py:29: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o", temperature=0.2, api_key=openai_api_key)
C:\Users\Admin\AppData\Local\Temp\ipykernel_8188\515078519.py:36: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output = chain.run(query)


Answer:
 The GE J47 is a turbojet engine developed by General Electric. It was one of the most widely produced jet engines in the United States and was used in several military aircraft during the 1950s and 1960s. The engine was known for its reliability and performance, contributing to its extensive use in various aircraft models. The J47 was also used in land speed record attempts, such as the one by Craig Breedlove in the Spirit of America, where it helped achieve a speed of 407.447 mph on August 5, 1963.
Output saved to LLM_Answer_Output.docx


In [28]:
import pandas as pd

def write_results_to_csv_df(results, query,filename="results_output.csv"):
    # Normalize list values for clean storage
    formatted_results = {}

    for key, value in results.items():
        if isinstance(value, list):
            # Convert list of items to a single string (join by separator)
            formatted_results[key] = ["\n---\n".join(str(v) for v in value)]
        else:
            formatted_results[key] = [str(value)]  # Wrap in list to make it a row

    # Create a DataFrame with one row and keys as columns
    df = pd.DataFrame.from_dict(formatted_results)
    df["query"]=query
    # Save to CSV
    df.to_csv(filename, index=False, encoding="utf-8")
    print(f"Results saved to {filename}")


In [29]:
write_results_to_csv_df(results,query, filename="results_output_table_q1.csv")

Results saved to results_output_table_q1.csv


Skipping Flat retriever test as the collection was not created.
HNSW Retriever Time: 0.6683s
HNSW Accuracy: 0.00%
hnsw Top Docs: [Document(metadata={'pk': 458312173011974298}, page_content='specific approach, leading to “efficient multitask serving.”\nPrompt design (prompt engineering):\n• The focus is on designing a very specific input or prompt to guide the pre-trained model to\nproduce the desired output.\n• Like prompt tuning, the main model remains “frozen”.\n• This method is about exploiting the vast knowledge and capabilities of the pre-trained model\nby just crafting the right input. As mentioned earlier, we will cover prompt engineering in\ndetail in Chapter 5.\nIn prompt tuning and prompt design, original model weights remain frozen, whereas in model tuning\nmodel parameters are updated:\n62 Fine-Tuning – Building Domain-Specific LLM Applications\nFigure 3.10 – Model tuning, prompt tuning, and prompt design\nThe following figure demonstrates model tuning (full fine-tuning) on

In [30]:
query = "Give me information about rocket?"

In [31]:
write_results_to_csv_df(results, query,filename="results_output_table_q2.csv")

Results saved to results_output_table_q2.csv


In [32]:
results = run_prompt_tuning_pipeline(
    query=query,
    milvus_flat=milvus_flat,
    milvus_hnsw=milvus_hnsw,
    milvus_ivf=milvus_ivf,
    milvus_connected=milvus_connected,
    openai_api_key=OPENAI_KEY
)


Skipping Flat retriever test as the collection was not created.
HNSW Retriever Time: 1.0294s
HNSW Accuracy: 0.00%
hnsw Top Docs: [Document(metadata={'pk': 458312173147910714}, page_content='576.553 Art Arfons Green Monster GE J79 11/7/65\n600.601 Craig Breedlove Spirit of America, Sonic 1 GE J79 11/15/65\n622.407 Gary Gabelich Blue Flame Rocket 10/23/70\n633.468 Richard Noble Thrust 2 RR RG 146 10/4/83\n763.035 Andy Green Thrust SSC RR Spey 10/15/97\nExample 5: Distance and Time (GR 8-10)\nThe following data were collected using a car with a water clock set to release a drop in\na unit of time and a meter stick. The car rolled down an inclined plane. Three trials were\nrun. Create a data table with an average distance column and an average velocity column,\ncreate an average distance-time graph, and draw the best-fit line or curve. Estimate the\ncar’s distance traveled and velocity at six drops of water. Describe the motion of the car. Is\nit going at a constant speed, accelerating, or

In [43]:
query="Give me diffrent ph vlues used by students?"

In [44]:
results = run_prompt_tuning_pipeline(
    query=query,
    milvus_flat=milvus_flat,
    milvus_hnsw=milvus_hnsw,
    milvus_ivf=milvus_ivf,
    milvus_connected=milvus_connected,
    openai_api_key=OPENAI_KEY
)


Skipping Flat retriever test as the collection was not created.
HNSW Retriever Time: 0.8157s
HNSW Accuracy: 0.00%
hnsw Top Docs: [Document(metadata={'pk': 458312173147910711}, page_content='ventilation, chemical-splash safety goggles, gloves, and apron. Do not use bleach,\nammonia, or strong acids with children.\nLemon juice 2.4, 2.0, 2.2; Baking soda (1 Tbsp) in Water (1 cup) 8.4, 8.3, 8.7;\nOrange juice 3.5, 4.0, 3.4; Battery acid 1.0, 0.7, 0.5; Apples 3.0, 3.2, 3.5;\nTomatoes 4.5, 4.2, 4.0; Bottled water 6.7, 7.0, 7.2; Milk of magnesia 10.5, 10.3,\n10.6; Liquid hand soap 9.0, 10.0, 9.5; Vinegar 2.2, 2.9, 3.0; Household bleach\n12.5, 12.5, 12.7; Milk 6.6, 6.5, 6.4; Household ammonia 11.5, 11.0, 11.5;\nLye 13.0, 13.5, 13.4; and Sodium hydroxide 14.0, 14.0, 13.9; Anti-freeze 10.1,\n10.9, 9.7; Windex 9.9. 10.2, 9.5; Liquid detergent 10.5, 10.0, 10.3; and\nCola 3.0, 2.5, 3.2\nTeaching tip: The pH scale is from 0 to 14. Have students make two data tables, one\nwith the data as given and o

In [45]:
write_results_to_csv_df(results, query,filename="results_output_table_q6.csv")

Results saved to results_output_table_q6.csv
